In [ ]:
import pandas as pd
import numpy as np
import random

In [ ]:
# exponential decay
x = 10
y = 5000*np.exp(-0.1*x)
y.round(2)

1839.4

In [ ]:
class WaterEnv:
  """When there is a rainfall, 40% of water from rain gets added into the lake,
  50% goes to river and 10% goes into ground water.
  Everyday 10% of lake water and 10% of river water goes into the groundwater
  due to seepage.

  Average daily consumption of water is 1500 lakh liters. Depending on
  increasing or decreasing price per liter, the consumption varies. As the price
  increases, consumption decreases.
  The relationship between consumption and price per liter is given by:

      `current_consumption = 5000 * np.exp(-0.1*current_price)`

  NOTE : Consumption and price follows an exponential decay relationship.
  As the price increases, the consumption decays exponentially.

  From the equation, current_price = 10 (rupees per liter)
  => current_consumption = 1839 (lakh liters per day).

  Amount of rainfall on each day can be set to either "low", "medium" or "high".
  This is assumed to be selected randomly or to be choosen dynamically based
  on the season or the area in which the simulation is being performed.

  The goal of the problem is to maximise the profits of the water supply board.
  Water consumption needs to be lesser when there is less rainfall and high when
  there is higher rainfall, so that the source does not get depleated and water
  continously gets consumed.
  --------------------------------------------------------------------------
  NOTE: Time iterval of the transitions are 1 day. T = 1, 2... n implies,
  1st day, 2nd day... nth day.

  Actions
  -------
  Every day, by looking at the water reserves in lake, river and groundwater,
  It is required to take a decision whether to: "increase price",
  "decrease price" or leave the price constant on that particular day.

  States
  ------
  {'ground_reserve': float,
  'lake_reserve': float,
  'river_reserve': float,
  'current_price': float}

  Rewards
  -------
  The income earned by selling water on that particular day OR number of
  liters consumed on a particular day.
  (When water gets over, the episode ends. Hence the agents would learn to
  maximise the consumption but not let the water reserve go empty)
  """
  def __init__(self):
    """
    Parameters
    ----------
    rainfall_level : str
      Can be "low", "medium" or "high"
    """
    self.avg_rainfall_levels = {
        "low": 10,
        "medium": 70,
        "high": 150
        }
    self.rainfall_level = "medium"
    self.avg_price = 10
    self.action_space = ["increase_price", "decrease_price", "constant_price"]

    self.reset()

  def reset(self):
    self.terminated = False
    # water reserve in each source measured in "lakh liters".
    self.state = {"ground_reserve":1500*20,
                  "lake_reserve":1500*20,
                  "river_reserve":1500*40,
                  "current_price":self.avg_price}

    self.total_reward = 0

  def set_rainfall_level(self, rainfall_level):
    self.rainfall_level = rainfall_level

  def rainfall_simulator(self, level):
    avg = self.avg_rainfall_levels[level]
    std = 5
    rainfall = np.random.normal(avg, 5)
    if rainfall<0:
      ranfall = 0
    return rainfall

  def get_consumption(self):
    # Consumption and price follows an exponential decay relationship.
    # As the price increases, the consumption decays exponentially.
    current_consumption = 5000 * np.exp(-0.1*self.state["current_price"])
    current_consumption = current_consumption.round()
    return current_consumption

  def update_states(self, rainfall):
    """Rainfall : 50% to river, 40% lake and 10% ground water
    10% from river and lake gets added to ground water.
    """
    terminated = False

    lake_reserve = self.state["lake_reserve"]
    ground_reserve = self.state["ground_reserve"]
    river_reserve = self.state["river_reserve"]

    current_consumption = self.current_consumption

    # Rainfall : 50% to river, 40% lake and 10% ground water
    lake_reserve += 0.4*rainfall
    ground_reserve += 0.1*rainfall
    river_reserve += 0.5*rainfall

    # 10% of lake reserve goes into ground water
    ground_reserve += 0.1*lake_reserve
    lake_reserve -= 0.1*lake_reserve

    # 10% of river reserve goes into ground water
    ground_reserve += 0.1*river_reserve
    river_reserve -= 0.1*river_reserve

    # account for current consumption.
    # half of daily consumption is met by river water and the other half by
    # ground water.
    # water for consumption is supplied from river and ground reserves.
    if ground_reserve + river_reserve < current_consumption:
      terminated = True
      return terminated
    if 0.5*current_consumption > ground_reserve:
      # Ground reserve is unable to meet half of the current day's consumption.
      # So, the rest in compensated by the river reserve.
      river_reserve = river_reserve - current_consumption + ground_reserve
      ground_reserve = 0
    elif 0.5*current_consumption > river_reserve:
      # River reserve is unable to meet half of the current day's consumption.
      # So, the rest in compensated by the ground reserve.
      ground_reserve = ground_reserve - current_consumption + river_reserve
      river_reserve = 0
    else:
      ground_reserve -= 0.5*current_consumption
      river_reserve -= 0.5*current_consumption

    # update the states
    self.state["lake_reserve"] = lake_reserve
    self.state["ground_reserve"] = ground_reserve
    self.state["river_reserve"] = river_reserve

    return terminated

  def get_reward(self):
    reward = self.state["current_price"] * self.current_consumption
    return reward

  def step(self, action):
    # Adjust the price according to the action taken
    if action == "increase_price":
      self.state["current_price"]+=1
    elif action == "decrease_price" and self.state["current_price"]>1:
      self.state["current_price"]-=1

    self.current_consumption = self.get_consumption()
    if self.current_consumption < 1:
      self.terminated = True
      reward = 0
      return self.state, reward, self.terminated

    rainfall = self.rainfall_simulator(self.rainfall_level)

    self.terminated = self.update_states(rainfall)

    reward = self.get_reward()

    self.total_reward+=reward

    return self.state, reward, self.terminated

### Random actions (random selection of pricing)

In [ ]:
# random actions

# rainfall level can be set to low, medium or high.
rainfall_level = "low"

env = WaterEnv()
env.reset()
env.set_rainfall_level(rainfall_level)

print(env.state)

for i in range(100000):
  action = random.choices(env.action_space)[0]
  state, reward, terminated = env.step(action)
  # print(state, reward, terminated)
  if terminated:
    print("Water reserves depleated")
    break

print(env.state)
print("Days sustained: ", i)
print("Total_reward: ", env.total_reward)

{'ground_reserve': 30000, 'lake_reserve': 30000, 'river_reserve': 60000, 'current_price': 10}
Water reserves depleated
{'ground_reserve': 1046.9141310685804, 'lake_reserve': 66.95355915609512, 'river_reserve': 0, 'current_price': 13}
Days sustained:  65
Total_reward:  1200893.0


### Always increase the price

In [ ]:
# Always increase price

# rainfall level can be set to low, medium or high.
rainfall_level = "low"

env = WaterEnv()

env.reset()
env.set_rainfall_level(rainfall_level)

for i in range(100000):
  action = "increase_price"
  state, reward, terminated = env.step(action)
  print(state, reward, terminated)
  if terminated:
    break

print("Days sustained: ", i)
print("Total_reward: ", env.total_reward)

{'ground_reserve': 38169.17488922471, 'lake_reserve': 27002.22610589945, 'river_reserve': 53170.78263237431, 'current_price': 11} 18304.0 False
{'ground_reserve': 45434.1270080966, 'lake_reserve': 24303.237433288577, 'river_reserve': 47102.24679161071, 'current_price': 12} 18072.0 False
{'ground_reserve': 51892.951771379856, 'lake_reserve': 21872.489914620768, 'river_reserve': 41709.99239327595, 'current_price': 13} 17719.0 False
{'ground_reserve': 57635.60561111071, 'lake_reserve': 19686.95681378408, 'river_reserve': 36924.638017230085, 'current_price': 14} 17262.0 False
{'ground_reserve': 62740.25076597082, 'lake_reserve': 17721.076089422142, 'river_reserve': 32677.692911777667, 'current_price': 15} 16740.0 False
{'ground_reserve': 67276.47862919501, 'lake_reserve': 15950.580831624751, 'river_reserve': 28907.439059530927, 'current_price': 16} 16144.0 False
{'ground_reserve': 71307.8231066641, 'lake_reserve': 14359.392726395283, 'river_reserve': 25565.032625994092, 'current_price': 17

### Always decrease the price

In [ ]:
# Always decrease price

# rainfall level can be set to low, medium or high.
rainfall_level = "low"

env = WaterEnv()

env.reset()
env.set_rainfall_level(rainfall_level)

for i in range(100000):
  action = "decrease_price"
  state, reward, terminated = env.step(action)
  if terminated:
    break

print("Days sustained: ", i)
print("Total_reward: ", env.total_reward)

Days sustained:  29
Total_reward:  217519.0


### Constant prices

In [ ]:
# Always decrease price

# rainfall level can be set to low, medium or high.
rainfall_level = "low"

env = WaterEnv()

env.reset()
env.set_rainfall_level(rainfall_level)

for i in range(100000):
  action = "constant_price"
  state, reward, terminated = env.step(action)
  if terminated:
    break

print("Days sustained: ", i)
print("Total_reward: ", env.total_reward)

Days sustained:  65
Total_reward:  1213740.0
